# Prepare Data

In [1]:
import os 
import sys
import random
import math
import numpy as np
import matplotlib.pyplot as plt
import json
from tqdm import tqdm, tqdm_notebook
import pandas as pd 
import time
import pdb
import seaborn as sns
import glob 
import traces
from functions.generators import data_generator,  one_hot_encode, create_XY
import pickle

# 

#make wider graphs
sns.set(rc={'figure.figsize':(12,5)})
plt.figure(figsize=(12,5))

# the following two lines are for changing imported functions, and not needing to restart kernel to use their updated version
%load_ext autoreload
%autoreload 2
%matplotlib inline

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

DATA_DIR = './Data/'
color_map = {0:'g', 1:'r', 2:'c', 3:'m', 4:'y', 5:'k'}
External_Dir='D:/DATA/'

Targets = np.array([ 6, 15, 16, 42, 52, 53, 62, 64, 65, 67, 88, 90, 92, 95,99])
real_targets =np.array([ 6, 15, 16, 42, 52, 53, 62, 64, 65, 67, 88, 90, 92, 95])

C:\Users\kiner\Anaconda3\envs\yuvals\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
df_training=pd.read_csv(DATA_DIR+'training_set.csv')
df_training_meta=pd.read_csv(DATA_DIR+'training_set_metadata.csv')
df_training.head()
df_training.shape
df_training_meta.head()
df_training_meta.shape

,object_id,mjd,passband,flux,flux_err,detected
0,615,59750.4229,2,-544.810303,3.622952,1
1,615,59750.4306,1,-816.434326,5.553370,1
2,615,59750.4383,3,-471.385529,3.801213,1
3,615,59750.4450,4,-388.984985,11.395031,1
4,615,59752.4070,2,-681.858887,4.041204,1


(1421705, 6)

,object_id,ra,decl,gal_l,gal_b,ddf,hostgal_specz,hostgal_photoz,hostgal_photoz_err,distmod,mwebv,target
0,615,349.046051,-61.943836,320.796530,-51.753706,1,0.0000,0.0000,0.0000,NaN,0.017,92
1,713,53.085938,-27.784405,223.525509,-54.460748,1,1.8181,1.6267,0.2552,45.4063,0.007,88
2,730,33.574219,-6.579593,170.455585,-61.548219,1,0.2320,0.2262,0.0157,40.2561,0.021,42
3,745,0.189873,-45.586655,328.254458,-68.969298,1,0.3037,0.2813,1.1523,40.7951,0.007,90
4,1124,352.711273,-63.823658,316.922299,-51.059403,1,0.1934,0.2415,0.0176,40.4166,0.024,90


(7848, 12)

In [3]:
df_training_meta.tail()

,object_id,ra,decl,gal_l,gal_b,ddf,hostgal_specz,hostgal_photoz,hostgal_photoz_err,distmod,mwebv,target
7843,130739978,26.718750,-14.940303,172.342697,-72.255675,0,0.0000,0.0000,0.0000,NaN,0.013,65
7844,130755807,120.101349,-62.696659,275.742955,-16.509746,0,0.1725,2.5606,1.1146,46.6108,0.136,90
7845,130762946,203.108109,-55.682144,308.728904,6.727511,0,0.0000,0.0000,0.0000,NaN,0.430,16
7846,130772921,79.101562,-35.501846,239.172243,-33.827844,0,0.0000,0.0000,0.0000,NaN,0.034,65
7847,130779836,301.992188,-17.426323,25.102988,-24.511101,0,0.0000,0.0000,0.0000,NaN,0.091,6


In [4]:
validate_p=0.25
df_train_meta=df_training_meta.head(0)
df_validate_meta=df_training_meta.head(0)
for target in real_targets:
    df = df_training_meta[df_training_meta.target==target]
    df_tr = df.sample(frac=1-validate_p)
    _,sl_vl = df.object_id.align(df_tr.object_id,join='left')
    df_vl=df[sl_vl.isna()] 
    df_train_meta=pd.concat([df_train_meta,df_tr])
    df_validate_meta=pd.concat([df_validate_meta,df_vl])
df_train_meta=df_train_meta.sample(frac=1)
df_validate_meta=df_validate_meta.sample(frac=1)
df_train_meta.shape[0]
df_validate_meta.shape[0]
df_training_meta.shape[0]

5885

1963

7848

In [5]:
df_train_meta.to_csv(DATA_DIR+'training_meta.csv',index=False)
df_validate_meta.to_csv(DATA_DIR+'validation_meta.csv',index=False)

## 4fold split

In [57]:
foldn=4
df_group=[]
df_group_meta=[]
df_validate_meta=[]
df_train_meta=[]
df_train_l=[]
for i in range(foldn):
    df_group.append([])
    df_group_meta.append([])
    df_validate_meta.append(None)
    df_train_meta.append(None)
    df_train_l.append([])
    
df_training_metas=df_training_meta.sample(frac=1).copy()
for target in real_targets:
    df = df_training_metas[df_training_metas.target==target].copy()
    for i in range(foldn):
        df_group[i].append(df.iloc[int(i*df.shape[0]/foldn):int(((i+1)*df.shape[0])/foldn)])
        
for i in range(foldn):
    df_group_meta[i]=pd.concat(df_group[i])


for i in range(foldn):
    for j in range(foldn):
        if (i==j):
            df_validate_meta[i]=df_group_meta[j].copy()
        else:
            df_train_l[i].append(df_group_meta[j].copy())
    
    df_train_meta[i]=pd.concat(df_train_l[i])
    df_train_meta[i].shape
    df_validate_meta[i].shape
    df_train_meta[i].to_csv('{}training_meta{}.csv'.format(DATA_DIR,i),index=False)
    df_validate_meta[i].to_csv('{}validation_meta{}.csv'.format(DATA_DIR,i),index=False)


(5892, 12)

(1956, 12)

(5884, 12)

(1964, 12)

(5887, 12)

(1961, 12)

(5881, 12)

(1967, 12)

In [55]:
df_group[0][]

,object_id,ra,decl,gal_l,gal_b,ddf,hostgal_specz,hostgal_photoz,hostgal_photoz_err,distmod,mwebv,target
5508,78242149,141.152344,-4.480799,237.106471,30.943970,0,0.0,0.0,0.0,NaN,0.035,6
6576,102464956,83.320312,-12.788582,215.939185,-23.029661,0,0.0,0.0,0.0,NaN,0.125,6
5918,87175959,242.578125,-32.619843,344.361443,13.818555,0,0.0,0.0,0.0,NaN,0.273,6
5135,69629671,99.492188,-1.641510,212.858577,-3.744959,0,0.0,0.0,0.0,NaN,1.739,6
7761,128983448,172.089844,-11.719760,272.801095,46.234873,0,0.0,0.0,0.0,NaN,0.027,6
2795,15797803,226.933594,-36.795330,331.166423,18.494352,0,0.0,0.0,0.0,NaN,0.071,6
3233,25094602,194.074890,-47.553680,303.782355,15.310040,0,0.0,0.0,0.0,NaN,0.122,6
7685,127374613,74.707031,-19.471222,219.371612,-33.222569,0,0.0,0.0,0.0,NaN,0.040,6
3831,39233485,244.335938,-35.501846,343.343991,10.746520,0,0.0,0.0,0.0,NaN,0.606,6
3984,42606035,99.667969,-9.594068,220.070675,-7.172270,0,0.0,0.0,0.0,NaN,0.526,6


In [51]:
df_group[2][4]

2

In [3]:
df_test_meta=pd.read_csv(DATA_DIR+'test_set_metadata.csv')

In [5]:
df_test_meta.shape

(3492890, 11)

In [12]:
test_point=[]
chunksize = 10000000
for i,chunk in tqdm(enumerate(pd.read_csv(DATA_DIR+'test_set.csv', chunksize=chunksize))):
    if i==0:
        extra = pd.DataFrame(chunk.head(0))
    l=chunk.shape[0]
    last=chunk.iloc[-1].object_id
    if l==chunksize:
        next_extra=chunk[chunk.object_id==last]
        chunk=chunk[chunk.object_id!=last]
    df=pd.concat([extra,chunk])
    test_point.append((df.iloc[0].object_id,df.iloc[-1].object_id))
    df.to_csv('{}test_set_p{}.csv'.format(External_Dir,i),index=False)
    del df
    extra=next_extra.copy()
    print(df_test_meta[df_test_meta.object_id==last].index.values[0]/df_test_meta.shape[0])



0it [00:00, ?it/s]

0.008680490940167025



1it [01:10, 70.24s/it]

0.8223313645720306



2it [02:24, 72.46s/it]

...


44it [57:06, 77.87s/it]

0.793715232944639



45it [58:22, 77.83s/it]

0.8018852583390831



46it [58:50, 76.75s/it]


In [14]:
df=pd.DataFrame(test_point,columns=['start','end'])
df.to_csv(External_Dir+'test_dict.csv',index=False)

In [27]:
object_ids=[]
for i in tqdm(range(46)):
    df=pd.read_csv('{}test_set_p{}.csv'.format(External_Dir,i))
    obj_ids=df.object_id.unique().astype('uint64').tolist()
    assert (len(set(obj_ids).intersection(set(object_ids)))==0), 'repeated object id:{}'.format(set(obj_ids).intersection(set(object_ids)))
    object_ids.extend(obj_ids)
   




  0%|                                                                                           | 0/46 [00:00<?, ?it/s]


  2%|█▊                                                                                 | 1/46 [00:06<04:47,  6.38s/it]


  4%|███▌                                                                               | 2/46 [00:12<04:41,  6.41s/it]


  7%|█████▍                                                                             | 3/46 [00:19<04:40,  6.52s/it]


  9%|███████▏                                                                           | 4/46 [00:26<04:34,  6.54s/it]


 11%|█████████                                                                          | 5/46 [00:33<04:32,  6.65s/it]


 13%|██████████▊                                                                        | 6/46 [00:39<04:26,  6.66s/it]


 15%|████████████▋                                                                      | 7/46 [00:47<04:22,  6.72s/it]


 17%|██████████████▍ 

In [28]:
len(object_ids)

3492890

In [29]:
df_test_meta.shape

(3492890, 11)

In [3]:
df_test_meta=pd.read_csv(DATA_DIR+'test_set_metadata.csv')
df_test_meta.shape
for t in range(46):
    print (t)
    df_timeseries=pd.read_csv('{}test_set_p{}.csv'.format(External_Dir,t))
    obj_ids=df_timeseries.object_id.unique().astype('uint64')
    df_test_meta_p=df_test_meta[df_test_meta.object_id.isin(obj_ids)]
    test_timeseries,test_meta = create_XY(df_timeseries,df_test_meta_p,num_samples=256,detected=False,weight=0,return_type='test')
    with open('{}timeseries0_p{}.pickle'.format(External_Dir,t), 'wb') as handle:
        pickle.dump(test_timeseries, handle, protocol=pickle.HIGHEST_PROTOCOL)


(3492890, 11)

35


100%|████████████████████████████████████████████████████████████████████████████| 78110/78110 [49:54<00:00, 26.09it/s]


34


100%|████████████████████████████████████████████████████████████████████████████| 78116/78116 [50:26<00:00, 25.81it/s]


In [4]:
df_test_meta=pd.read_csv(DATA_DIR+'test_set_metadata.csv')
df_test_meta.shape
for t in tqdm_notebook(range(46)):
    df_timeseries=pd.read_csv('{}test_set_p{}.csv'.format(External_Dir,t))
    obj_ids=df_timeseries.object_id.unique().astype('uint64')
    df_test_meta_p=df_test_meta[df_test_meta.object_id.isin(obj_ids)]
    test_timeseriese = create_XY(df_timeseries,df_test_meta_p,Targets=real_targets,num_samples=256,return_type='error')
    with open('{}timeseriese_p{}.pickle'.format(External_Dir,t), 'wb') as handle:
        pickle.dump(test_timeseriese, handle, protocol=pickle.HIGHEST_PROTOCOL)


(3492890, 11)

HBox(children=(IntProgress(value=0, max=46), HTML(value='')))

HBox(children=(IntProgress(value=0, max=30320), HTML(value='')))

HBox(children=(IntProgress(value=0, max=74021), HTML(value='')))

HBox(children=(IntProgress(value=0, max=78126), HTML(value='')))

...

HBox(children=(IntProgress(value=0, max=78170), HTML(value='')))

HBox(children=(IntProgress(value=0, max=78127), HTML(value='')))

HBox(children=(IntProgress(value=0, max=78130), HTML(value='')))

HBox(children=(IntProgress(value=0, max=78112), HTML(value='')))

HBox(children=(IntProgress(value=0, max=28538), HTML(value='')))

In [43]:
(df_mins.max_mean_predp<1.3).sum()

4821

In [56]:
df_test_extras=[]
for t in tqdm(range(46)):
    df_test_timeseries=pd.read_csv('{}test_set_p{}.csv'.format(External_Dir,t))
    gp=df_test_timeseries.groupby('object_id')
    dfe=(gp['mjd'].max()-gp['mjd'].min()).rename('dmjd').reset_index()
    dfe['ratio']=gp.detected.mean().rename('ratio').reset_index().ratio
    df_test_extras.append(dfe)

100%|██████████████████████████████████████████████████████████████████████████████████| 46/46 [09:35<00:00, 12.50s/it]


In [57]:
df_test_extra=pd.concat(df_test_extras)

In [58]:
df_test_extra.head()

,object_id,dmjd,ratio
0,13,853.8455,0.157576
1,14,853.8455,0.012121
2,17,873.7903,0.014205
3,23,853.8455,0.018182
4,34,873.7903,0.065341


In [61]:
df_test_extra.to_csv(DATA_DIR+'test_extra.csv',index=False)